# 데이터셋(Dataset)
- **Train 데이터셋 (훈련/학습 데이터셋)**
    - 모델을 학습시킬 때 사용할 데이터셋.
- **Validation 데이터셋 (검증 데이터셋)**
    - 모델의 성능 중간 검증을 위한 데이터셋
- **Test 데이터셋 (평가 데이터셋)**
    - 모델의 성능을 최종적으로 측정하기 위한 데이터셋
    - **Test 데이터셋은 마지막에 모델의 성능을 측정하는 용도로 한번만 사용되야 한다.**
        - 모델을 훈련하고 평가했을때 원하는 성능이 나오지 않으면 데이터나 모델 학습을 위한 설정(하이퍼파라미터)을 수정한 뒤에 다시 훈련시키고 평가를 하게 된다. 원하는 성능이 나올때 까지 설정변경->훈련->평가를 반복하게 된다. 
        - 위 사이클을 반복하게 되면 평가결과를 바탕으로 설정을 변경하게 되므로 모델이 평가할 때 사용한 데이터셋(Test set)에 모델이 맞춰서 훈련하는 것과 동일한 효과를 내게 된다.(설정을 변경하는 이유가 Test set에 대한 결과를 좋게 만들기 위해 변경하므로) 그래서 Train dataset과 Test dataset 두개의 데이터셋만 사용하게 되면 모델의 성능을 제대로 평가할 수 없게 된다. 그래서 데이터셋을 train 세트, validation 세터, test 세트로 나눠 train set 와 validation set으로 모델을 최적화 한 뒤 마지막에 test set으로 최종 평가를 한다.

# Data 분리 방식

## Hold Out
- 데이터 셋을 Train set, Validation set, Test set으로 나눈다.
- sklearn.medel_selection.train_test_split() 함수 사용
    - 하나의 dataset을 2 분할 해 주는 함수

### Hold out 방식의 단점
- train/validation/test 셋이 어떻게 나눠 지냐에 따라 결과가 달라진다.
    - 데이터가 충분히 많을때는 변동성이 흡수되 괜찮으나 적을 경우 문제가 발생할 수 있다.
        - 이상치에 대한 영향을 많이 받는다.
        - 다양한 패턴을 찾을 수가 없기 때문에 새로운 데이터에 대한 예측 성능이 떨어지게 된다.
        
- **Hold out 방식은 (다양한 패턴을 가진) 데이터의 양이 많을 경우에 사용한다.**

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
X, y = load_iris(return_X_y=True)   #return_X_y=True: Feature와 target(label) 배열만 tuple에 묶어서 반환.
print(X.shape, y.shape)

(150, 4) (150,)


### Train/Test set 분리

In [3]:
X_train, X_test, y_train, y_test= train_test_split(X, # Feature(Input, X)
                                                   y, # Target(Label, Output, y)
                                                   test_size=0.3, # test 셋의 비율
                                                   random_state=0,# random seed 값 => shuffle을 random하게 한다.
                                                   stratify=y)
                                                   # **분류문제**에서 분할된 데이터셋의 class별 데이터수의 비율을 원본데이터셋과 동일하게 만들어준다.
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(105, 4) (45, 4) (105,) (45,)


### Train/Validation/Test set 분리

In [4]:
# 원본 Dataset => Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

# Train set => Train/Validation 분할
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, #Train set
                                                  test_size=0.2, stratify=y_train, random_state=0)
X_train.shape, X_val.shape, X_test.shape

((96, 4), (24, 4), (30, 4))

### 모델생성, 평가
하이퍼파라미터(hyper parameter) 튜닝(tuning)
- max_depth=1 => 0.66666
- max_depth=2 => 1

> 하이퍼파라미터: 모델생성할때 우리가 설정해주는 설정값으로 모델 성능에 영향을 준다.
> 하이퍼파라미터 튜닝: 모델의 성능이 좋아지도록 하이퍼파라미터 값을 변경하는 것.

In [6]:
tree = DecisionTreeClassifier(random_state=0, max_depth= 2) 

# 학습-train
tree.fit(X_train, y_train)

# 평가-validation
pred_val = tree.predict(X_val)
val_acc = accuracy_score(y_val, pred_val)

print("val 정확도: ", val_acc)

val 정확도:  1.0


### test set으로 최종검증

In [7]:
test_pred = tree.predict(X_test)
accuracy_score(y_test, test_pred)

0.9333333333333333

## K-겹 교차검증 (K-Fold Cross Validation) 
1. 데이터셋을 설정한 K 개로 나눈다.
1. K개 중 하나를 검증세트로 나머지를 훈련세트로 하여 모델을 학습시키고 평가한다. 
1. K개 모두가 한번씩 검증세트가 되도록 K번 반복하여 모델을 학습시킨 뒤 나온 평가지표들을 평균내서 모델의 성능을 평가한다.
- 데이터양이 충분치 않을때 사용한다.
- 보통 Fold를 나눌때 2.5:7.5 또는 2:8 비율이 되게 하기 위해 4개 또는 5개 fold로 나눈다. 
- 종류
    - **KFold**
        - 회귀문제의 Dataset을 분리할 때 사용
    - **StratifiedKFold**
        - 분류문제의 Dataset을 분리할 때 사용

### KFold
- 지정한 개수(K)만큼 분할한다.
- Raw dataset의 순서를 유지하면서 지정한 개수로 분할한다.
- 회귀 문제일때 사용한다.
- KFold(n_splits=K)
    - 몇개의 Fold로 나눌지 지정
- KFold객체.split(데이터셋)
    - 데이터셋을 지정한 K개 나눴을때 train/test set에 포함될 데이터의 **index**들을 반환하는 generator 생성 

In [8]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold   #KFold 클래스 - train_test_split()와 동일한 모듈
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X, y = load_iris(return_X_y=True)

In [11]:
# 각 fold별 검증 결과들을 담을 리스트
acc_train_list = [] # trainset으로 평가한 결과
acc_test_list = []  # test dataset으로 평가한 결과

# KFold 객체 생성 - 3개 fold
kfold = KFold(n_splits=3)

for train_index, test_index in kfold.split(X):   #split() index를 제공하는 generator생성 => 튜플 (train set index,  test set index)
    
    # KFold generator가 제공한 index를 이용해 data 조회
    X_train, y_train = X[train_index], y[train_index] #train set 추출
    X_test, y_test = X[test_index], y[test_index]     #test set 추출
    
    # 모델생성
    tree = DecisionTreeClassifier()
    
    # 학습
    tree.fit(X_train, y_train)
    
    # 추론
    pred_train = tree.predict(X_train)
    pred_test = tree.predict(X_test)
    
    # 평가 - 정확도 
    acc_train = accuracy_score(y_train, pred_train)  #(정답, 예측값)
    acc_test = accuracy_score(y_test, pred_test)
    acc_train_list.append(acc_train)  #평가결과들을 list에 append(추가)
    acc_test_list.append(acc_test)

In [12]:
acc_train_list, np.mean(acc_train_list)

([1.0, 1.0, 1.0], 1.0)

In [13]:
acc_test_list, np.mean(acc_test_list) 
# 정확도 0.0 : 한개도 못맞췄다. ===> 분류문제일 경우 KFold(회귀)를 사용하지 말고 StratifiedKFold를 사용해야 한다.

([0.0, 0.0, 0.0], 0.0)

### StratifiedKold
- 나뉜 fold 들에 label들이 같은(또는 거의 같은) 비율로 구성 되도록 나눈다. 
- 각각의 클래스 별로 각각 순서대로 나눈다.
- 분류문제일 때 사용한다.
- StratifiedKold(n_splits=K)
    - 몇개의 Fold로 나눌지 지정
- StratifiedKold객체.split(X, y)
    - 데이터셋을 지정한 K개 나눴을때 train/test set에 포함될 데이터의 index들을 반환하는 generator 생성
    - input(X)와 output(y) dataset을  전달한다. 

In [14]:
from sklearn.model_selection import StratifiedKFold

X, y = load_iris(return_X_y=True)

In [15]:
# 폴더별 검증 결과들을 담을 list
acc_train_list = []
acc_test_list = []

s_fold = StratifiedKFold(n_splits=3)

for train_index, test_index in s_fold.split(X, y):  # 반복-튜플 (train index,  test index)
    
    # train/test 데이터셋 추출
    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
    
    # 모델생성
    tree = DecisionTreeClassifier()
    
    # 학습
    tree.fit(X_train, y_train)
    
    # 추론
    pred_train = tree.predict(X_train)
    pred_test = tree.predict(X_test)
    
    # 평가
    acc_train = accuracy_score(y_train, pred_train)
    acc_test = accuracy_score(y_test, pred_test)
    
    acc_train_list.append(acc_train)
    acc_test_list.append(acc_test)


In [16]:
print(acc_train_list, np.mean(acc_train_list))

[1.0, 1.0, 1.0] 1.0


In [17]:
print(acc_test_list, np.mean(acc_test_list))

[0.98, 0.92, 1.0] 0.9666666666666667


## cross_val_score( )
- 데이터셋을 K개로 나누고 K번 반복하면서 평가하는 작업을 처리해 주는 함수
- 주요매개변수
    - estimator: 모델객체
    - X: feature(input data)
    - y: label(output data)
    - scoring: 평가지표
    - cv: 나눌 개수 (K)
        - int: 개수
        - KFold, StratifiedKFold 객체
- 반환값: array - 각 반복마다의 평가점수    

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score  
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=0)

data = iris_data.data
label = iris_data.target

# scores = cross_val_score(dt_clf , data , label , scoring='accuracy', cv=3)
# kfold = KFold(n_splits=3)
kfold = StratifiedKFold(n_splits=3)
scores = cross_val_score(dt_clf , data , label , scoring='accuracy', cv=kfold)  
print(scores)
print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

[0.98 0.94 0.98]
교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667
